In [ ]:
#Adding the pandas pack to our data
import pandas as pd
import re

# Load aliexpress pet supplier data- each "title" row corresponds to a single product 
df = pd.read_csv('aliexpress_pet_supplies.csv')

#Clean data 
# Remove 'sold', '+', as well as any alphabetic characters from the data (specifically the column 'tradeAmount')
df['tradeAmount'] = df['tradeAmount'].astype(str).str.replace(r'sold|\+|[A-Za-z]', '', regex=True)

# Remove any extra spaces and converting column to a numeric character (int or float)
df['tradeAmount'] = df['tradeAmount'].str.strip()
df['tradeAmount'] = pd.to_numeric(df['tradeAmount'], errors='coerce')

# df['tradeAmount'] contains only numbers after cleaning 

# Calculate Turnover Ratio and Conversion Rate, my two key metrics that we will calculate and utilize to measure and rank our suppliers
df['Turnover Ratio'] = df['quantity'] / df['tradeAmount'] #amouunt sold/ current quantity in inventory
df['Conversion Rate'] = df['tradeAmount'] / df['wishedCount'] #amount sold/ current wished count 

# Standardize each metric from 0-100, giving each score a rating on the same initial plane
df['Star Rating Score'] = (df['averageStar'] / 5) * 100
df['Turnover Score'] = (df['Turnover Ratio'] / df['Turnover Ratio'].max()) * 100
df['Conversion Score'] = (df['Conversion Rate'] / df['Conversion Rate'].max()) * 100

# Weighted final score, asscociated weights are as follows
#Star rating encompasses 40% of my final supplier score, Turnover score 35%, and Conversion score 25%. 
#Weights of course reflect my personal judgement and opinions in terms of my analysis and could misconstrue the totality of our suppliers viability. This completely depends on the business you run, the weights are subjective.
df['Final Score'] = (
    df['Star Rating Score'] * 0.4 +
    df['Turnover Score'] * 0.35 +
    df['Conversion Score'] * 0.25
)

# Setting final score to 0 if 'tradeAmount' is 0 to remove bias and error asscociated with companies that have 0 sales (inheritely untrustworthy) 
df.loc[df['tradeAmount'] == 0, 'Final Score'] = 0

#To measure our suppliers (based on the above criteria)
# Calculate the 90th percentile threshold
threshold = df['Final Score'].quantile(0.9)

# Filter for my top 10% suppliers
top_10_percent = df[df['Final Score'] >= threshold]

# Sort by Final Score 'descending' to cleanly distinguish our best suppliers
top_10_percent = top_10_percent.sort_values(by='Final Score', ascending=False)

# Save to Excel or print
top_10_percent.to_excel('aliexpress_top_10_percent.xlsx', index=False)
print(top_10_percent)

print(df[['title', 'Star Rating Score', 'Turnover Ratio', 'Conversion Rate', 'Final Score']])